### Importing our Libraries

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [8]:
#setting our columns to display full values
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:20,.2f}'.format)

### Installing important libraries

In [9]:
#i had to install this to work on this excel file 
!pip install xlrd

Defaulting to user installation because normal site-packages is not writeable


### Reading our Dataset and Column Descriptions to our notebook


In [10]:
#importing the data
df = pd.read_excel("Week2_challenge_data_source.xlsx")

In [11]:
#importing the data description
df_descript = pd.read_csv("Field Descriptions - Sheet1.csv")

FileNotFoundError: [Errno 2] File Field Descriptions - Sheet1.csv does not exist: 'Field Descriptions - Sheet1.csv'

Removing all columns with space and lowering the  letters of the columns

In [ ]:
df.columns = [label.replace(' ', '_').lower() for label in df.columns]

### Exploring our Datasets

I want to try to understand the data here and check for missing values 

We start with understanding what different columns stand for, here is a view of the different columns meaning this helps others understand this data too 

In [ ]:
df_descript

This shows us the different data types and number of non-null values in each columns

In [ ]:
df.info()

This shows us the Numbers of Rows and Columns in our dataset 

In [ ]:
df.shape 

The numbers of unique values in each columns are shown below to understand a little bit, how the data is distributed 

In [ ]:
df.nunique().sort_values(ascending=False)

### Analysis of Null Values in our dataset

I want to analyze my dataset to understand where the null values are and how to deal with them 

In [ ]:
#i am trying to understanding the null values here by analyzing the percentage of missing values in each columns
null = df.isna().sum().sort_values(ascending = False)
null_per = (null/150001) * 100
null_perc = pd.DataFrame(null_per)
null_perc.head(10)

I have noticed the columns with a lot of missing values and would decide now how to impute this, i would be mean as already stated for numerical columns and median to replace null for Categorical columns 

I would start with getting all the missing numerical values in our missing data, i would turn all the values in null percent to a list of columns  

In [ ]:
null_perc.reset_index(inplace=True)
missing = list(null_perc['index'])

We would append numerical values that are missing to one list and the categorical columns to another list

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numcols =[]
catcols = []
for data in missing:
    if df[data].dtypes in numerics:
        numcols.append(data)
    else:
        catcols.append(data)    

There are some values that are numerical but are not supposed to be like the IMEI, IMSI,MSISDN we would remove them from our list of numerical values

we would create a new list for the categorical values remove them from the numerical list and add to the categorical columns

In [ ]:
catcoladd =['imei', 'imsi','msisdn/number']
for c in catcoladd:
    numcols.remove(c)

In [ ]:
catcols.extend(catcoladd)

We would start with filling the missing values in the numerical column with the mean of the column

In [ ]:
for col in numcols:
    df[col] = df[col].fillna(df[col].mean())

Confirm if what we did work

In [ ]:
df[numcols].isnull().any()

Next we would try to fill the categorical variables by the mode since we cannot use mean here

we would have to convert all the new columns we added to object data type

In [ ]:
df[catcoladd] = df[catcoladd].astype('object')

In [ ]:
for col in catcols:
    df[col] = df[col].fillna(df[col].mode()[0])

we would check all our categorical column to see if what we did worked 

In [ ]:
df[catcols].isnull().any()

### User Overview Analysis

#### Identifying the top 10 handsets used by the customers.

In [ ]:
top_ten = df['handset_type'].value_counts(ascending = False).head(10).reset_index()
top_ten = top_ten.rename(columns={'index':"Handset Type","handset_type":"Count"})
top_ten

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x = top_ten['Handset Type'], y = top_ten['Count'])
plt.xlabel('Name of handsets')
plt.xticks(rotation=45)
plt.title('A barplot showing the top ten handset types');

#### Top 3 Handset Manufacturer

In [ ]:
top_three = df['handset_manufacturer'].value_counts(ascending = False).head(3).reset_index()
top_three = top_three.rename(columns={'index':"Handset Manufacturer","handset_manufacturer":"Count"})
top_three

#### Top 5 handset per Handset Manufacturer

In [ ]:
top_fivemanf = df.groupby(['handset_manufacturer','handset_type'])['handset_type'].count().reset_index(name="count")

In [ ]:
top_fivemanf=top_fivemanf.sort_values(['handset_manufacturer', 'count']).groupby('handset_manufacturer').head(5)
top_fivemanf

####  Make a short interpretation to the marketing team 


     Top ten handset

    From the list of top ten Handset we can observe that the most purchase unique Handset is *Huawei B528S-23A* is our most used handset while Apple IPhone is the most purchased in our Top ten Handset, Samsung also appear in the list as one of the top ten phone
    

     Recommendations
        we should try to make our service tailored for this brands.

#### Overview of Users Behavior


Bearer ID according to our data description is unique for each session, the bearer ID is our unique session identifier so to identify each session we would the bearer ID here to group them, and count them to know the total number of Unique sessions

In this analysis we would be using MSISDN as a unique identifier for each users

####  number of xDR sessions

In [ ]:
df['bearer_id'].value_counts()

    There is a misnomer in the first row in our Bearer ID
    I dropped all rows without Bearer ID, since we
    cannot confirm if they had more than one session or not

the total number of unique Session in our dataset is the number of unique Bearer ID

In [ ]:
bearer_id = pd.DataFrame(df[['msisdn/number','bearer_id']])
bearer_id= bearer_id.loc[bearer_id['bearer_id'] != '']

In [ ]:
NumberOfSessions=bearer_id.groupby('msisdn/number').agg({'bearer_id': 'count'}).reset_index()
NumberOfSessions=NumberOfSessions.rename(columns= {'bearer_id': 'xDR sessions'})

In [ ]:
NumberOfSessions.head()

In [ ]:
plt.subplots(figsize=(10, 6))
sns.distplot(NumberOfSessions["xDR sessions"], hist=True )

#### Session Duration

In [ ]:
SessionDuration =df.groupby('msisdn/number').agg({'dur._(ms).1': 'sum'}).reset_index()
#here is the duration for first five rows in our dataset
SessionDuration=SessionDuration.rename(columns= {'dur._(ms).1': 'sessions duration(ms)'})
SessionDuration.head()

In [ ]:
SessionDuration['dur._(ms).1'].describe()

In [ ]:
sns.boxplot(SessionDuration["dur._(ms).1"])

In [ ]:
plt.hist(SessionDuration["dur._(ms).1"])

In [ ]:
SessionDuration.describe().T

In [ ]:
UpDownload_user = df[['msisdn/number','total_ul_(bytes)','total_dl_(bytes)']]
UpDownload_user = UpDownload_user.groupby('msisdn/number').sum().reset_index()
UpDownload_user.drop(['msisdn/number'], axis=1, inplace=True)
UpDownload_user.describe()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30, 10))
ax[0].hist(total_updown);
sns.boxplot(UpDownload_user["Applications UL/DL (bytes)"],ax=ax[1]);
ax[0].title.set_text('Histogram showing the Total  Data Usage by all Applications in Bytes')
ax[1].title.set_text('Box Plot showing the Total Data Usage by all Applications in Bytes')
ax[2].title.set_text('Distribution Plot showing the Total Data Usage  by all Applications in Bytes')
sns.distplot(total_updown["Applications UL/DL (bytes)"], ax=ax[2]);
fig.savefig('Applications.jpeg')

In [ ]:
applicationDownUp =df[['social_media_dl_(bytes)','social_media_ul_(bytes)', 'google_dl_(bytes)',
                        'google_ul_(bytes)', 'email_dl_(bytes)', 'email_ul_(bytes)', 'youtube_dl_(bytes)',
                        'youtube_ul_(bytes)','netflix_dl_(bytes)', 'netflix_ul_(bytes)', 'gaming_dl_(bytes)',
                        'gaming_ul_(bytes)','other_dl_(bytes)', 'other_ul_(bytes)']]
total_updown = pd.DataFrame()
total_updown["social_media_total(bytes)"] = df["social_media_dl_(bytes)"] + df['social_media_ul_(bytes)']
total_updown["google_total(bytes)"] = df["google_dl_(bytes)"] + df["google_ul_(bytes)"]
total_updown['email_total(bytes)'] = df["email_dl_(bytes)"] + df["email_ul_(bytes)"]
total_updown['youtube_total(bytes)'] = df["youtube_dl_(bytes)"] + df["youtube_ul_(bytes)"]
total_updown['netflix_total(bytes)'] = df["netflix_dl_(bytes)"] + df["netflix_ul_(bytes)"]
total_updown["gaming_total(bytes)"] = df["gaming_dl_(bytes)"] + df["gaming_ul_(bytes)"]
total_updown['other_total(bytes)']=df["other_dl_(bytes)"]+df["other_ul_(bytes)"]

In [ ]:
total_updown["Total Applications (bytes)"]= total_updown['social_media_total(bytes)'] + total_updown['google_total(bytes)']+ total_updown['email_total(bytes)']+ total_updown['youtube_total(bytes)']+ total_updown['netflix_total(bytes)']+ total_updown['gaming_total(bytes)']+ total_updown['other_total(bytes)']

In [ ]:
total_updown.describe().T

In [ ]:
new = pd.DataFrame(total_updown.sum(axis=0, skipna=True))

In [ ]:
new =new.reset_index()

In [ ]:
new= new.rename(columns={'index':"Applications",0:"Sum"})

In [ ]:
new = new[:7]


In [ ]:
new

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.barplot(x="Applications", y="Sum", data=new)
plt.xticks(rotation=45)
plt.title("A Bar plot showing the data consumption for each Applications",fontsize = 12, fontweight= "bold")
plt.xticks(fontsize = 10, fontweight= "bold")
plt.tight_layout()
plt.savefig("totaL.jpeg")

In [ ]:
total_updown["Total Applications (bytes)"] = total_updown["Total Applications (bytes)"].astype("int16")

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30, 10))
ax[0].hist(total_updown);
sns.boxplot(total_updown["Total Applications (bytes)"],ax=ax[1]);
ax[0].title.set_text('Histogram showing the Total  Data Usage by all Applications in Bytes')
ax[1].title.set_text('Box Plot showing the Total Data Usage by all Applications in Bytes')
ax[2].title.set_text('Distribution Plot showing the Total Data Usage  by all Applications in Bytes')
sns.distplot(total_updown["Total Applications (bytes)"], ax=ax[2]);
fig.savefig('Applications.jpeg')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30, 10))
ax[0].hist(total_updown["Total Applications (bytes)"]);
sns.boxplot(total_updown["Total Applications (bytes)"], ax=ax[1]);
ax[0].title.set_text('Histogram showing the Total  Data Usage by all Applications in Bytes')
ax[1].title.set_text('Box Plot showing the Total Data Usage by all Applications in Bytes')
ax[2].title.set_text('Distribution Plot showing the Total Data Usage  by all Applications in Bytes')
sns.distplot(total_updown["Total Applications (bytes)"], ax=ax[2]);
fig.savefig('Applications.jpeg')

In [ ]:
print("Kurtosis for Duration of Call is", pd.DataFrame(total_updown["email_total(bytes)"]).kurtosis()[0], "Skewedness of Duration of call is",
      pd.DataFrame(total_updown["email_total(bytes)"]).skew()[0],
    sep='\n'
)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30, 10))
ax[0].hist(total_updown["youtube_total(bytes)"]);
sns.boxplot(total_updown["youtube_total(bytes)"], ax=ax[1]);
ax[0].title.set_text('Histogram showing the Total  Data Usage on Youtube in Bytes')
ax[1].title.set_text('Box Plot showing the Total Data Usage on Youtube in Bytes')
ax[2].title.set_text('Distribution Plot showing the Total Data Usage on Youtube in Bytes')
sns.distplot(total_updown["youtube_total(bytes)"], ax=ax[2]);
fig.savefig('Youtube.jpeg')

In [ ]:
print("Kurtosis for the total Upload/Download is", pd.DataFrame(total_updown["youtube_total(bytes)"]).kurtosis()[0], "Skewedness of Duration of call is",
      pd.DataFrame(total_updown["youtube_total(bytes)"]).skew()[0],
    sep='\n'
)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30, 10))
ax[0].hist(total_updown["netflix_total(bytes)"]);
sns.boxplot(total_updown["netflix_total(bytes)"], ax=ax[1]);
ax[0].title.set_text('Histogram showing the Total Netflix Data Usage in Bytes')
ax[1].title.set_text('Box Plot showing the Total Netflix Data Usage in Bytes')
ax[2].title.set_text('Distribution Plot showing the Total Netflix Data Usage in Bytes')
sns.distplot(total_updown["netflix_total(bytes)"], ax=ax[2]);
fig.savefig('netflix.jpeg')